In [10]:
import pandas as pd
import numpy as np

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import ast

In [2]:
ner_data = pd.read_csv('ner_extracted_2.csv', index_col=0)
ner_data = ner_data.drop_duplicates(subset='filename')
ner_data.reset_index(drop=True, inplace=True)
ner_data

,filename,error,court,petitioners,respondents,judges,date,org,gpe,provisions,statutes,precedents
0,Abhijeet_Suryakant_Maske_And_Anr_vs_The_State_...,NaN,"['High Court Of Judicature At Bombay', 'Suprem...","['Abhijeet Suryakant Maske', 'Abhijeet Suryaka...","['State Of Maharashtra', 'State of Maharashtra...","['C.V. Bhadang', 'C.V. Bhadang', 'C.V. Bhadang...","['1 March, 2022', '6 November 2020', '21 Septe...","['Baramati Police Station, Dist', 'Baramati Po...","['Pune', 'Mumbai', 'Mumbai', 'Pune']","['Section 328 of Indian Penal Code', 'Section ...","['Indian Penal Code', 'Food Safety and Standar...",['Abhijeet Suryakant Maske And Anr vs The Stat...
1,Amluya_Anand_vs_State_on_8_March_2022.PDF,'provision_statute_pairs',['High Court Of Judicature For Rajasthan At\n ...,"['Amluya Anand', 'Amluya Anand']","['State', 'State']","['Sandeep Mehta', 'Sandeep Mehta', 'Merta', 'C...","['08/03/2022', '17/02/2022', '07.04.2015', '24...","['Swastik Milk Chilling Centre', 'Amul Dairy',...","['Nagaur', 'Thanwla District Nagaur', 'Jodhpur...","['Section 468', 'Section 77', 'Sections 26(2)(...","['Cr', 'Food Safety and Standards Act, 2006', ...","['CRLMP-545/2020]Amluya Anand vs State', 'V.V...."
2,Arulmurugan_Starch_Industries_vs_Food_Safety_A...,NaN,['High Court Of Judicature At Madras'],"['Arulmurugan Starch Industries', '2018Arulmur...","['Food Safety And Standards', 'Commissioner of...","['Krishnan Ramasamy', 'Krishnan Ramasamy', 'KR...","['11.7.2018', '19.01.2022', '19 January, 2022'...",['Food Safety and Standards Authority of India'],"['New Delhi', 'Chennai', 'Salem District', 'Sa...",[],[],[]
3,Arun_Raosaheb_Khot_vs_State_Of_Maharashtra_on_...,NaN,"['High Court Of Judicature At Bombay', 'Suprem...","['Arun Raosaheb Khot', 'Sneha\nNitin', 'Arun R...","['State Of Maharashtra', 'State of Maharashtra']","['C.V. Bhadang', 'C.V. Bhadang', 'C.V. Bhadang...","['31 January, 2022', '6 November 2020', '21 Se...","['Police Station Hadapsar, District Pune']","['Mumbai', 'Mumbai']",['Section 328 of Indian Penal Code'],['Indian Penal Code'],[]
4,Asha_Singh_vs_The_State_Of_Bihar_on_24_March_2...,NaN,"['High Court Of Judicature At Patna', 'Patna H...","['Asha Singh', 'Asha Singh']","['State Of Bihar', 'State of Bihar', 'Union Of...","['Ashutosh Kumar', 'Ashutosh Kumar']","['24-03-2022', 'dt.24-03-2022', '24 March, 202...","['C.B.I.', 'Tata Institute of Social Sciences'...","['Turkauliya', 'Raghunathpur', 'Motihari', 'Bi...","['Section 341 of Criminal Procedure Code', 'Se...","['Criminal Procedure Code', 'Protection of Chi...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...
189,Vijaysing_Dnyaneshwar_Gotya_S_O_vs_State_Of_Mh...,'provision_statute_pairs',"['High Court Of Judicature At Bombay', 'Additi...","['Vijaysing @ Dnyaneshwar @ Gotya', 'Vijaysing...","['State Of Mha', 'State of\n ...","['Avinash G. Gharote', 'AVINASH G. GHAROTE', '...","['25.11.2021', '24.11.21', '25.11.2021', '13 J...","['German Cancer Research Centre (DKFZ)', 'Depa...","['Malkapur-Kurha', 'Harsoda', 'State of Mahara...","['Sections 188, 272, 273, 328', 'Sections 26(2...","['IPC', 'Food Safety and Standard Act 2006', '...",['Vijaysing @ Dnyaneshwar @ Gotya S/O ... vs S...
190,Viky_Vikash_Ku_Agrawal_vs_State_Of_Odisha_Opp_...,NaN,['High Court Of Orissa At Cuttack'],"['Viky @ Vikash Ku Agrawal', 'Viky @ Vikash Ku...","['State Of Odisha', 'State of Odisha']","['S.K. Sahoo', 'S.K. Sahoo', 'S.K. Sahoo', 'J....","['11.01.2021', '9 February, 2022', '13.04.2021...",[],[],['section 272/273/420/269/270 of Indian Penal ...,"['Indian Penal Code', 'Cr.', 'Food Safety and ...",[]
191,Vishnu_Gupta_Lalla_vs_The_State_Of_Madhya_Prad...,NaN,"['High Court Of Madhya Pradesh', 'Supreme Court']","['Vishnu Gupta @ Lalla', 'Vishnu Gupta @ Lalla']","['State Of Madhya Pradesh', 'State Of Madhya P...","['Anjuli Palo', 'ANJULI PALO']","['10.8.2021', '22.1.2021', '3 January, 2022', ...","['P.S.\nRanjhi', 'Khadya Suraksha Aur Manak Ad...",['Jabalpur'],"['Section 420 of Indian Penal Code', 'Section ...","

In [3]:
gpt_data = pd.read_csv('openai_extracted.csv', index_col=0)
gpt_data = gpt_data[gpt_data['file_name'].isin(ner_data['filename'])]
gpt_data

,file_name,summary,court,petitioners,respondents,judges,date,org,gpe,provisions,statutes,precedents,key_facts,type_of_case,decision
0,Abhijeet_Suryakant_Maske_And_Anr_vs_The_State_...,In the case of Abhijeet Suryakant Maske and An...,HIGH COURT OF JUDICATURE AT BOMBAY,"['Abhijeet Suryakant Maske', 'Dilip Ramchandra...",['The State of Maharashtra'],['C.V. Bhadang'],01-03-2022,[],"['Pune', 'Maharashtra']","['Section 328 of IPC', 'Section 272 of IPC', '...","['Food Safety and Standards Act, 2006']","['Joseph Kurian & Anr. Vs. State of Kerala', '...",The Applicants are seeking anticipatory bail i...,Anticipatory Bail Application,in favour of appellant
1,Amluya_Anand_vs_State_on_8_March_2022.PDF,**Case Summary: Amluya Anand vs State (8 March...,HIGH COURT OF JUDICATURE FOR RAJASTHAN AT JODHPUR,['Amluya Anand'],"['State, Through Vinod Sharma, Food Safety Off...",['Sandeep Mehta'],08-03-2022,"['Amul Dairy', 'Food Safety Officer', 'Chief M...","['Mujjafarpur', 'Gurgaon', 'Nagaur', 'Devgarh ...","['Section 468 of Cr.P.C.', 'Section 77 of the ...","['Food Safety and Standards Act, 2006']",['Sri.V.V.S.S.R. Prakash Rao vs. The State of ...,The petitioner sought to quash orders taking c...,Criminal Miscellaneous Petition,in favour of appellant
2,Arulmurugan_Starch_Industries_vs_Food_Safety_A...,**Case Summary: Arulmurugan Starch Industries ...,High Court of Judicature at Madras,['Arulmurugan Starch Industries'],['Food Safety and Standards Authority of India...,['Krishnan Ramasamy'],19-01-2022,['Food Safety and Standards Authority of India...,"['Salem District', 'New Delhi', 'Chennai']",['Article 226 of the Constitution of India'],"['Food Safety and Standards Act, 2006']",[],The petitioner filed a writ petition challengi...,Writ Petition,The court directed the Commissioner of Food Sa...
3,Arun_Raosaheb_Khot_vs_State_Of_Maharashtra_on_...,In the case of Arun Raosaheb Khot vs. State of...,High Court of Judicature at Bombay,['Arun Raosaheb Khot'],['The State of Maharashtra'],['C.V. Bhadang'],31-01-2022,[],"['Pune', 'Maharashtra']","['Section 272 of IPC', 'Section 273 of IPC', '...","['Food Safety and Standards Act, 2006']",['Joseph Kurian & Anr. Vs. State of Kerala'],The applicant is seeking anticipatory bail in ...,Anticipatory Bail Application,in favour of appellant
4,Asha_Singh_vs_The_State_Of_Bihar_on_24_March_2...,In the case of Asha Singh vs The State of Biha...,HIGH COURT OF JUDICATURE AT PATNA,['Asha Singh'],"['The State of Bihar', 'THE UNION OF INDIA THR...",['Ashutosh Kumar'],24-03-2022,"['Tata Institute of Social Sciences', 'SAKHI']","['Bihar', 'Patna', 'East Champaran', 'Motihari']","['Section 341 of IPC', 'Section 342 of IPC', '...","['Indian Penal Code', 'Protection of Children ...",[],"The petitioner, Asha Singh, is seeking anticip...",Criminal Miscellaneous,against appellant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Vijaysing_Dnyaneshwar_Gotya_S_O_vs_State_Of_Mh...,**Case Summary: Vijaysing @ Dnyaneshwar @ Goty...,"High Court of Judicature at Bombay, Nagpur Bench",['Vijaysing @ Dnyaneshwar @ Gotya Raghunandan ...,['State of Maharashtra'],['Avinash G. Gharote'],13-01-2022,[],"['Maharashtra', 'Mumbai']","['Section 188 of IPC', 'Section 272 of IPC', '...","['Food Safety and Standard Act 2006', 'Cigaret...",['Ganesh Pandurang Jadhao vrs. State of Mahara...,The applicant was arrested for transporting co...,Criminal Application (Bail),against appellant
192,Viky_Vikash_Ku_Agrawal_vs_State_Of_Odisha_Opp_...,In the case of Viky @ Vikash Ku Agrawal vs Sta...,High Court of Orissa at Cuttack,['Viky @ Vikash Ku Agrawal'],['State of Odisha'],['S.K. Sahoo'],09-02-2022,[],"['Odisha', 'Kantabanji']","['Section 438 of Cr.P.C.', 'Section 272 of IPC...","['Indian Penal Code', 'Food Safety and Standar...",[],The petitioner applied for anticipatory bail i...,Anticipatory bail application,in favour of appellant
193,Vishnu_Gupta_Lalla_vs_The_State_Of_Madhya_Prad...,"On January 3, 2022, the High Court of Madhya P...",High Cour

In [ ]:
gpt_data['statutes'] = gpt_data['statutes'].apply(lambda x: ast.literal_eval(x))
ner_data['statutes'] = ner_data['statutes'].apply(lambda x: ast.literal_eval(x))

In [59]:
gpt_data['judges'] = gpt_data['judges'].apply(lambda x: ast.literal_eval(x))
ner_data['judges'] = ner_data['judges'].apply(lambda x: ast.literal_eval(x))

In [55]:

gpt_data['provisions'] = gpt_data['provisions'].apply(lambda x: ast.literal_eval(x))
ner_data['provisions'] = ner_data['provisions'].apply(lambda x: ast.literal_eval(x))

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def fuzzy_deduplicate(statutes, threshold=80):
    deduplicated_statutes = []

    for statute in statutes:
        statute = statute.strip()  

        # Skip if statute is empty
        if not statute:
            continue

        # Check if the statute is similar to any already in the deduplicated list
        if deduplicated_statutes:
            match, score = process.extractOne(statute, deduplicated_statutes, scorer=fuzz.ratio)
            if score >= threshold:
                # Statute is considered a duplicate, skip adding it
                continue

        # If no match above the threshold is found, add it to the deduplicated list
        deduplicated_statutes.append(statute)

    return deduplicated_statutes


In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def calculate_metrics_statutes(llm_df, ner_df, threshold=80):
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    # Iterate through each row (case) in the DataFrames
    for i in range(len(llm_df)):
        llm_statutes = llm_df.iloc[i]['statutes'] if isinstance(llm_df.iloc[i]['statutes'], list) else []
        ner_statutes = ner_df.iloc[i]['statutes'] if isinstance(ner_df.iloc[i]['statutes'], list) else []

        ner_statutes = fuzzy_deduplicate(ner_statutes, threshold=60)
        
        # Track matches for each case
        case_true_positives = 0
        case_false_positives = 0
        case_false_negatives = 0

        # If LLM statutes are non-empty, perform matching
        if llm_statutes:
            for llm_statute in llm_statutes:
                llm_statute = llm_statute.strip()  # Remove leading/trailing spaces

                # Skip if llm_statute is an empty string after stripping
                if not llm_statute:
                    continue

                # Use fuzzy matching to find the best match in NER statutes
                match, score = process.extractOne(llm_statute, ner_statutes, scorer=fuzz.ratio) if ner_statutes else (None, 0)

                if score >= threshold:
                    case_true_positives += 1  # Statute matches within the threshold
                else:
                    case_false_positives += 1  # No good match found
                # if i % 5 == 0 and score < threshold and score > 20:
                #     print(f"{llm_statute} ---- {match} ---- {score}")
        # Calculate false negatives (entities present in NER but not matched in LLM)
        case_false_negatives = len(ner_statutes) - case_true_positives

        # Update global counts
        true_positives += case_true_positives
        false_positives += case_false_positives
        false_negatives += case_false_negatives

    # Precision: True Positives / (True Positives + False Positives)
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0

    # Recall: True Positives / (True Positives + False Negatives)
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

    # F1 Score: 2 * (Precision * Recall) / (Precision + Recall)
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score
    }

out2 = calculate_metrics_statutes(gpt_data, ner_data, threshold=45)

In [71]:
out2

{'precision': 0.7559241706161137,
 'recall': 0.48114630467571645,
 'f1_score': 0.5880184331797235}

In [74]:
thresholds = range(10, 101, 5)
for threshold in thresholds:
    out = calculate_metrics_statutes(gpt_data, ner_data, threshold=threshold)
    print(f"Threshold: {threshold}, Precision: {out}")


Threshold: 10, Precision: {'precision': 0.943127962085308, 'recall': 0.6003016591251885, 'f1_score': 0.7336405529953917}
Threshold: 15, Precision: {'precision': 0.9028436018957346, 'recall': 0.5746606334841629, 'f1_score': 0.7023041474654379}
Threshold: 20, Precision: {'precision': 0.8767772511848341, 'recall': 0.5580693815987934, 'f1_score': 0.6820276497695852}
Threshold: 25, Precision: {'precision': 0.8720379146919431, 'recall': 0.555052790346908, 'f1_score': 0.6783410138248849}
Threshold: 30, Precision: {'precision': 0.8317535545023697, 'recall': 0.5294117647058824, 'f1_score': 0.6470046082949309}
Threshold: 35, Precision: {'precision': 0.7985781990521327, 'recall': 0.5082956259426847, 'f1_score': 0.6211981566820277}
Threshold: 40, Precision: {'precision': 0.7843601895734598, 'recall': 0.49924585218702866, 'f1_score': 0.6101382488479262}
Threshold: 45, Precision: {'precision': 0.7559241706161137, 'recall': 0.48114630467571645, 'f1_score': 0.5880184331797235}
Threshold: 50, Precision